In [11]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer('/home/rayguo17/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/7dbbc90392e2f80f3d3c277d6e90027e55de9125/')
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

embeddings = model.encode(sentences)
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-1.37173478e-02 -4.28515598e-02 -1.56286508e-02  1.40537526e-02
  3.95537764e-02  1.21796302e-01  2.94333510e-02 -3.17523964e-02
  3.54959555e-02 -7.93140009e-02  1.75878629e-02 -4.04369794e-02
  4.97259982e-02  2.54912637e-02 -7.18700141e-02  8.14968422e-02
  1.47069572e-03  4.79627252e-02 -4.50335778e-02 -9.92174670e-02
 -2.81769522e-02  6.45046458e-02  4.44670469e-02 -4.76217195e-02
 -3.52952294e-02  4.38671745e-02 -5.28566018e-02  4.33042354e-04
  1.01921454e-01  1.64072271e-02  3.26996483e-02 -3.45986746e-02
  1.21339289e-02  7.94871226e-02  4.58342955e-03  1.57778244e-02
 -9.68205929e-03  2.87626348e-02 -5.05806208e-02 -1.55793969e-02
 -2.87907235e-02 -9.62281600e-03  3.15556526e-02  2.27349531e-02
  8.71449336e-02 -3.85027453e-02 -8.84718671e-02 -8.75495002e-03
 -2.12343317e-02  2.08923928e-02 -9.02078226e-02 -5.25732301e-02
 -1.05638830e-02  2.88311355e-02 -1.61454882e-02  6.17839862e-03
 -1.23234

In [12]:
import pandas as pd

df = pd.read_parquet('../wiki_dataset/wiki_parquet/00000010.parquet',engine='pyarrow')
print(df.shape)
print(df.iloc[1])


(2000, 3)
index                                                     12
title                                              Anarchism
article    {{short description|Political philosophy and m...
Name: 1, dtype: object


In [13]:

df['embedding']=df['article'].apply(model.encode)
df.head()

,index,title,article,embedding
0,10,AccessibleComputing,#REDIRECT [[Computer accessibility]]\n\n{{rcat...,"[0.025729092, 0.015573813, -0.016550384, -0.02..."
1,12,Anarchism,{{short description|Political philosophy and m...,"[-0.044421196, 0.028122878, -0.0075959656, -0...."
2,13,AfghanistanHistory,#REDIRECT [[History of Afghanistan]]\n\n{{Redi...,"[0.11619787, 0.05068159, -0.000114921415, 0.04..."
3,14,AfghanistanGeography,#REDIRECT [[Geography of Afghanistan]]\n\n{{Re...,"[0.1423226, 0.04584206, 0.0061261063, 0.026503..."
4,15,AfghanistanPeople,#REDIRECT [[Demographics of Afghanistan]]\n\n{...,"[0.14802513, 0.0537404, 0.010204663, 0.0740573..."


In [15]:
len(df['embedding'].iloc[0])

384

In [24]:
import sqlite3

conn = sqlite3.connect("test.db")

for index,row in df.iterrows():
    conn.execute("INSERT INTO wiki_embedding VALUES (? , ? , ?)",[row['index'],row['title'],row['embedding'].tobytes()])
    break
#
conn.commit()
conn.close()